In [1]:
import os
import json
import numpy as np
from omegaconf import OmegaConf

In [2]:
exp_dir = "../../../logs/en/asr/evaluate"
folders = {
    "Fine-tuned": "version_9",
    "AC_100": "version_44",
    "AC_10": "version_45",
    "AC_1": "version_24",
    "AC_0.1": "version_46",
    "AC_0.01": "version_47",
}

In [3]:
# define which accent is standard, which are seen and which are unseen
classes = {
    "seen": ["us", "uk", "ca", "in", "de", "au"],
    "unseen": ["hk", "sg", "sc", "nz", "ie", "za", "ni", "ph"],
}

In [4]:
# load WERs of each file, for each accent
wers = dict()
for exp, folder in folders.items():
    wers[exp] = {k[5:-4]: v for k, v in json.load(open(f"{exp_dir}/{folder}/avg_wers.json")).items()}

In [5]:
# transform dictionary into a numpy array, where one axis represents the accents and another the experiments
experiments = list(wers.keys())
accents = list(wers[experiments[0]].keys())
n_words = np.array([wers[experiments[0]][acc]["n_words"] for acc in accents])
avg_wers = np.array([[wers[exp][acc]["avg_wer"] for exp in experiments] for acc in accents])

In [6]:
# compute means for each class, for each experiment
class_indices = {k: [accents.index(acc) for acc in v] for k, v in classes.items()}
class_means = {k: np.mean(avg_wers[indices], axis=0) for k, indices in class_indices.items()}

In [7]:
# print the avg. WERs and the means (overall and per class) as a markdown table
headers = ["Accent / Dialect"] + experiments
print((" | ").join(headers))
print(f"|{'---:|'*len(headers)}")

# print avg. WERs
for i in range(len(accents)):
    row = [accents[i]] + [f"{avg_wers[i,j]*100:.2f}" for j in range(len(experiments))]
    print((" | ").join(row))

# print overall means
row = ["mean"] + [f"{np.mean(avg_wers[:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))

# print class means
for key, value in class_means.items():
    row = [f"{key} mean"] + [f"{value[i]*100:.2f}" for i in range(len(experiments))]
    print((" | ").join(row))

# print worst avg. WER of each experiment
row = ["worst"] + [f"{v*100:.2f}" for v in np.max(avg_wers, axis=0)]
print((" | ").join(row))

# print the avg. of the three worst avg. WERs of each experiment
row = ["3-worst mean."] + [f"{np.mean(np.sort(avg_wers, axis=0)[-3:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))


Accent / Dialect | Fine-tuned | AC_100 | AC_10 | AC_1 | AC_0.1 | AC_0.01
|---:|---:|---:|---:|---:|---:|---:|
za | 6.54 | 99.10 | 10.11 | 6.54 | 6.52 | 6.53
sg | 10.41 | 97.05 | 15.78 | 10.51 | 10.41 | 10.34
in | 9.21 | 98.17 | 13.24 | 9.41 | 9.18 | 9.19
au | 7.47 | 98.15 | 13.21 | 7.53 | 7.47 | 7.42
hk | 9.76 | 102.11 | 14.36 | 9.75 | 9.73 | 9.71
ie | 7.51 | 98.65 | 12.18 | 7.41 | 7.41 | 7.47
uk | 5.78 | 99.14 | 9.27 | 5.87 | 5.76 | 5.74
ph | 8.90 | 97.34 | 13.33 | 8.79 | 8.79 | 8.79
de | 5.62 | 95.08 | 8.34 | 5.66 | 5.57 | 5.58
ni | 6.96 | 94.68 | 10.55 | 7.02 | 6.86 | 6.93
us | 5.43 | 98.42 | 7.50 | 5.46 | 5.41 | 5.40
ca | 5.36 | 97.29 | 7.13 | 5.35 | 5.32 | 5.32
nz | 5.40 | 100.04 | 10.97 | 5.28 | 5.36 | 5.33
sc | 33.50 | 98.13 | 39.11 | 32.55 | 33.83 | 33.77
mean | 9.13 | 98.10 | 13.22 | 9.08 | 9.12 | 9.11
seen mean | 6.48 | 97.71 | 9.78 | 6.55 | 6.45 | 6.44
unseen mean | 11.12 | 98.39 | 15.80 | 10.98 | 11.12 | 11.11
worst | 33.50 | 102.11 | 39.11 | 32.55 | 33.83 | 33.77
3-worst m

Accent / Dialect | Fine-tuned | AC_100 | AC_10 | AC_1 | AC_0.1 | AC_0.01
|---:|---:|---:|---:|---:|---:|---:|
Southern African (South Africa, Zimbabwe, Namibia) | 6.54 | 99.10 | 10.11 | 6.54 | 6.52 | 6.53
Singaporean English | 10.41 | 97.05 | 15.78 | 10.51 | 10.41 | 10.34
India and South Asia (India, Pakistan, Sri Lanka) | 9.21 | 98.17 | 13.24 | 9.41 | 9.18 | 9.19
Australian English | 7.47 | 98.15 | 13.21 | 7.53 | 7.47 | 7.42
Hong Kong English | 9.76 | 102.11 | 14.36 | 9.75 | 9.73 | 9.71
Irish English | 7.51 | 98.65 | 12.18 | 7.41 | 7.41 | 7.47
England English | 5.78 | 99.14 | 9.27 | 5.87 | 5.76 | 5.74
Filipino | 8.90 | 97.34 | 13.33 | 8.79 | 8.79 | 8.79
German English,Non native speaker | 5.62 | 95.08 | 8.34 | 5.66 | 5.57 | 5.58
Northern Irish | 6.96 | 94.68 | 10.55 | 7.02 | 6.86 | 6.93
United States English | 5.43 | 98.42 | 7.50 | 5.46 | 5.41 | 5.40
Canadian English | 5.36 | 97.29 | 7.13 | 5.35 | 5.32 | 5.32
New Zealand English | 5.40 | 100.04 | 10.97 | 5.28 | 5.36 | 5.33
Scottish English | 33.50 | 98.13 | 39.11 | 32.55 | 33.83 | 33.77
mean | 9.13 | 98.10 | 13.22 | 9.08 | 9.12 | 9.11
seen mean | 6.48 | 97.71 | 9.78 | 6.55 | 6.45 | 6.44
unseen mean | 11.12 | 98.39 | 15.80 | 10.98 | 11.12 | 11.11
worst | 33.50 | 102.11 | 39.11 | 32.55 | 33.83 | 33.77
3-worst mean. | 17.89 | 100.43 | 23.08 | 17.61 | 17.99 | 17.94

In [8]:
# print the experiment folders of each experiment (both train and eval folders)
print("Experiment folders:\n")
for exp, folder in folders.items():
    eval_folder = os.path.join(exp_dir, folder)
    eval_config = OmegaConf.load(os.path.join(eval_folder, "config.yaml"))
    train_folder = f'../{eval_config.asr.ckpt.replace("/checkpoints/last.ckpt", "")}'
    print(f"- {exp}: training `{train_folder[3:]}`, evaluation `{eval_folder[3:]}`")

Experiment folders:

- Fine-tuned: training `logs/en/asr/train/version_11`, evaluation `../../logs/en/asr/evaluate/version_9`
- AC_100: training `logs/en/ensemble/train/binary/b7/DAT/version_86`, evaluation `../../logs/en/asr/evaluate/version_44`
- AC_10: training `logs/en/ensemble/train/binary/b7/DAT/version_87`, evaluation `../../logs/en/asr/evaluate/version_45`
- AC_1: training `logs/en/ensemble/train/binary/b7/DAT/version_66`, evaluation `../../logs/en/asr/evaluate/version_24`
- AC_0.1: training `logs/en/ensemble/train/binary/b7/DAT/version_88`, evaluation `../../logs/en/asr/evaluate/version_46`
- AC_0.01: training `logs/en/ensemble/train/binary/b7/DAT/version_89`, evaluation `../../logs/en/asr/evaluate/version_47`
